In [24]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
from torchvision.utils import save_image
import torch.optim as optim
import pathlib
import torch.nn.functional as F

In [2]:
DATAPATH = pathlib.Path.cwd() / "data"

model = models.vgg19(weights="IMAGENET1K_V1").features
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")


In [3]:
CONTENT_PATH =DATAPATH / "squareflower.jpg"

In [4]:
STYLE_PATH = DATAPATH /"picasso.jpg"

In [6]:
def image_loader(path):
    image = Image.open(path)
    loader = transforms.Compose(
        [
            transforms.Resize(
                (
                    256,
                    256,
                )
            ),
            transforms.ToTensor(),
        ]
    )
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


In [7]:
content_img = image_loader(CONTENT_PATH) 
style_img = image_loader(STYLE_PATH)

In [8]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.req_features = ["0", "5", "10", "19", "28"]
        # Since we need only the 5 layers in the model so we
        #  will be dropping all the rest layers from the features of the model
        self.model = models.vgg19(weights="IMAGENET1K_V1").features[
            :29
        ]  # model will contain the first 29 layers

    # x holds the input tensor(image) that will be feeded to each layer
    def forward(self, x):
        # initialize an array that wil hold the
        # activations from the chosen layers
        features = []
        # Iterate over all the layers of the mode
        for layer_num, layer in enumerate(self.model):
            # activation of the layer will stored in x
            x = layer(x)
            # appending the activation of the selected
            #  layers and return the feature array
            if str(layer_num) in self.req_features[0:]:
                features.append(x)
                
        return features


In [9]:
# Load the model to the GPU
model = VGG().to(device).eval()

In [10]:
style_feat = model(style_img)
cont_feat=model(content_img)
gen_image = content_img.clone().detach().requires_grad_(True)
gen_feat = model(gen_image)

In [ ]:
len(style_feat)

In [25]:
def calc_style_loss(gen, style):
    # Calculating the gram matrix for the style and the generated image
    style_l=0
    for x in range(0,5,1):
        #print(x)
        batch, channel, height, width = gen[x].shape
        genitem = gen[x]
        styleitem = style[x]
        G = torch.mm(
            genitem.view(
             channel,
            height * width,
        ),
        genitem.view(channel, height * width).t(),
        )
        A = torch.mm(
        styleitem.view(
            channel,
            height * width,
        ),
        styleitem.view(channel, height * width).t(),
        )

    # Calcultating the style loss of each layer by
    # calculating the MSE between the gram matrix of
    # the style image and the generated image and adding it to style loss
        style_l += F.mse_loss(G,A)
    return style_l


In [ ]:
gen_feat[3].shape

In [ ]:
test1 = calc_style_loss(gen_feat, style_feat)

In [ ]:
test1

In [29]:
def calc_content_loss(gen_feat, orig_feat):
    # calculating the content loss of each layer
    #  by calculating the MSE between the content and
    #  generated features and adding it to content loss
    content_l = 0
    x=4
    #for x in range(len(gen_feat)):
    content_l += F.mse_loss(gen_feat[x], orig_feat[x])
    return content_l



In [ ]:
test = calc_content_loss(gen_feat, cont_feat)

In [ ]:
test2 = calc_style_loss(gen_feat, style_feat)

In [ ]:
test2

In [ ]:
test

In [13]:
alpha = 1000
beta = 10

In [14]:
def calculate_loss(
    gen,
    cont,
    style,
):
    style_loss = content_loss = 0
        # extracting the dimensions from the generated image
    content_loss += calc_content_loss(gen, cont)
    style_loss += calc_style_loss(gen, style)

    # calculating the total loss of e th epoch
    total_loss = alpha * content_loss + beta * style_loss
    return total_loss


In [ ]:
test3 = calculate_loss(gen_feat,cont_feat,style_feat)

In [ ]:
test3

In [15]:
epoch = 1000
lr = 0.04
alpha = 1000
beta = 50



In [30]:
def image_trainer() -> Image:
    original_image = image_loader(DATAPATH / "squareflower.jpg")
    style_image = image_loader(DATAPATH / "YellowFlowerWaterPixelTrim.jpg")
    image_name = "gen5.jpg"
    generated_image = original_image.clone().requires_grad_(True)
    model.requires_grad_(False)
    # using adam optimizer and it will update
    #  the generated image not the model parameter
    optimizer = optim.Adam([generated_image], lr=lr)

    # iterating for 1000 times
    for e in range(100):
        # extracting the features of generated, content
        # and the original required for calculating the loss
        gen_features = model(generated_image)
        orig_features = model(original_image)
        style_features = model(style_image)
        
        # iterating over the activation of each layer
        # and calculate the loss and add it to the content and the style loss
        total_loss = calculate_loss(
            gen_features,
            orig_features,
            style_features,
        )
        # optimize the pixel values of the generated
        # image and backpropagate the loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        # print the image and save it after each 100 epoch
        if e / 100:
            print(total_loss)

            save_image(generated_image, image_name)


In [31]:
try1 = image_trainer()

tensor(6.1845e+08, grad_fn=<AddBackward0>)
tensor(5.7175e+08, grad_fn=<AddBackward0>)
tensor(6.0897e+08, grad_fn=<AddBackward0>)
tensor(5.8398e+08, grad_fn=<AddBackward0>)
tensor(5.3986e+08, grad_fn=<AddBackward0>)
tensor(4.8806e+08, grad_fn=<AddBackward0>)
tensor(4.3959e+08, grad_fn=<AddBackward0>)
tensor(3.9659e+08, grad_fn=<AddBackward0>)
tensor(3.5789e+08, grad_fn=<AddBackward0>)
tensor(3.2335e+08, grad_fn=<AddBackward0>)
tensor(2.9241e+08, grad_fn=<AddBackward0>)
tensor(2.6518e+08, grad_fn=<AddBackward0>)
tensor(2.4060e+08, grad_fn=<AddBackward0>)
tensor(2.1888e+08, grad_fn=<AddBackward0>)
tensor(1.9926e+08, grad_fn=<AddBackward0>)
tensor(1.8143e+08, grad_fn=<AddBackward0>)
tensor(1.6574e+08, grad_fn=<AddBackward0>)
tensor(1.5220e+08, grad_fn=<AddBackward0>)
tensor(1.4031e+08, grad_fn=<AddBackward0>)
tensor(1.2949e+08, grad_fn=<AddBackward0>)
tensor(1.1944e+08, grad_fn=<AddBackward0>)
tensor(1.1055e+08, grad_fn=<AddBackward0>)
tensor(1.0395e+08, grad_fn=<AddBackward0>)
tensor(9866